In [ ]:
import re
from pathlib import Path

import pandas as pd
from IPython.display import display
from sklearn.model_selection import train_test_split

In [ ]:
RANDOM_SEED = 23
MIMIC_DIR = Path.cwd().parent / "data" / "physionet.org" / "files"

MIMIC_III_DIR = MIMIC_DIR / "mimiciii" / "1.4"
MIMIC_IV_DIR = MIMIC_DIR / "mimiciv" / "2.2" / "note"

In [ ]:
df = pd.read_csv(MIMIC_III_DIR / "NOTEEVENTS.csv")

In [ ]:
len(df["HADM_ID"].unique())

In [ ]:
len(df)

In [ ]:
df = df[df["ISERROR"] != 1]
df.drop("ISERROR", axis=1, inplace=True)
df = df.drop_duplicates()
df.head()

In [ ]:
df["CATEGORY"].unique()

In [ ]:
select_df = df[df["CATEGORY"].isin(["Discharge summary", "Physician "])]
select_df.head()

In [ ]:
grouped_df = select_df.groupby("HADM_ID")

In [ ]:
def clean_text(text: str) -> str:
    cleaned_text = re.sub(r"\n{3,}", "\n\n", text)
    cleaned_text = re.sub(r"\n {2,}", "\n", cleaned_text)
    return cleaned_text


grouped_df = grouped_df.filter(lambda group: len(group["CATEGORY"].unique()) == 2)
grouped_df["TEXT"] = grouped_df["TEXT"].apply(lambda text: clean_text(text))
grouped_df = grouped_df.groupby("HADM_ID")

In [ ]:
train_keys, test_keys = train_test_split(
    list(grouped_df.groups.keys()), test_size=0.2, random_state=RANDOM_SEED
)

In [ ]:
train_dfs = {key: grouped_df.get_group(key) for key in train_keys}

In [ ]:
train_summaries = [
    df[df["CATEGORY"] == "Discharge summary"]["TEXT"].values[0]
    for df in train_dfs.values()
]
len(train_summaries)

In [ ]:
sample = next(iter(train_dfs.values()))
sample

In [ ]:
sample = next(iter(train_dfs.values()))
sample = sample.sort_values("CHARTDATE")
display(sample)  # type: ignore
for _, row in sample.iterrows():
    print("=" * 80)
    print(row["CATEGORY"])
    print(row["TEXT"])
    print()

In [ ]:
re.findall(r"\n[^0-9\n]+:\n", train_summaries[0])

In [ ]:
print(train_summaries[0])

In [ ]:
sample_group = grouped_df.get_group(train_keys[1]).sort_values("CHARTDATE")
print(len(sample_group))
for _, row in sample_group.iterrows():
    print("=" * 80)
    print(row["CATEGORY"])
    print(row["TEXT"])

In [ ]:
for key in train_keys[:10]:
    # print(key)
    group = grouped_df.get_group(key)
    discharge_summary = group[group["CATEGORY"] == "Discharge summary"]
    print("=" * 80)
    print(discharge_summary["TEXT"].iloc[0])

In [ ]:
discharge_summaries = 


titles = 